In [1]:
import pandas as pd

### 서울대병원 데이터 크롤링 [(링크)](http://www.snuh.org/health/nMedInfo/nList.do?searchNWord=&sortType=A)

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


def snu_crowling(disease, idx, df):
    chrome_option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_option)
    
    # 서울대학교 병원 홈페이지
    driver.get('http://www.snuh.org/health/nMedInfo/nList.do?searchNWord=&sortType=A')

    # 검색 버튼 클릭
    search_btn = driver.find_element(By.XPATH, '//*[@id="gnb"]/button[2]')
    search_btn.click()
    time.sleep(1)

    # 검색창에 질병명 입력 및 전송
    search_ipt = driver.find_element(By.XPATH, '//*[@id="totSearchInp"]')
    search_ipt.send_keys(disease)
    search_ipt.send_keys(Keys.RETURN)
    time.sleep(1)

    # 새로 열린 검색 결과 탭으로 이동
    driver.switch_to.window(driver.window_handles[-1])

    # 검색 결과 페이지의 내비게이션 바에서 "건강정보" 클릭
    navi_btn = driver.find_element(By.XPATH, '//*[@id="gnb"]/ul/li[4]/a')
    navi_btn.click()
    time.sleep(1)
    
    more_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/a')
    more_btn.click()
    time.sleep(1)
    
    sort_acc = driver.find_element(By.XPATH, '//*[@id="rank"]')
    sort_acc.click()
    time.sleep(1)
    
    results_list = driver.find_element(By.CLASS_NAME, 'thumbType02').find_elements(By.CLASS_NAME, 'item')
    elem_list = []
    title_list = []
    for result in results_list:
        elem_list.append(result.find_element(By.TAG_NAME, 'strong'))
        title_list.append(result.find_element(By.TAG_NAME, 'strong').text.replace(" ", ""))
    
    best = []
    for title in title_list:
        if '[N의학정보]' + disease.replace(" ", "") in title:
            best.insert(0, title_list.index(title))
        elif '[N의학정보]' in title and disease.replace(" ", "") in title:
            best.append(title_list.index(title))
        
    elem_list[best[0]].click()
        

    # 새로 열린 질환 세부 정보 페이지로 이동
    driver.switch_to.window(driver.window_handles[-1])
    
    
    # 질환 세부 정보 페이지의 목차
    contents = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[3]/div[1]/div/ul').find_elements(By.TAG_NAME, 'li')
    contents = [content.text for content in contents]
    
    
    # 질환 세부 정보 페이지의 목차와 xpath를 매칭
    xpath_dict = dict.fromkeys(contents)
    
    for i in range(len(contents)):
        xpath_dict[contents[i]] = f'//*[@id="content"]/div[2]/div[3]/div[2]/div[{i+1}]/p'
    

    cols = list(df.columns)
    # ['질환명', '영문명', '증상 키워드', '정의', '증상', '원인', '관련신체기관',
    #                            '진단', '검사', '치료', '경과/합병증', '생활가이드', 'URL']
    
    
    # 질환명은 따로 수집하여, 파싱을 통해 국문명과 영문명을 추출
    name = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/h3').text
    kr_name = name.split('\n')[0]
    en_name = name.split('\n')[1].replace('[','').replace(']','')   
    df.loc[idx, '질환명'] = kr_name
    df.loc[idx, '영문명'] = en_name
    
    
    # 증상 키워드 수집
    df.loc[idx, '증상 키워드'] = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[2]/div[4]/p').text
    
    
    # 질환 세부 정보 수집
    err_list = []
    try:
        for info, xpath in xpath_dict.items():
            if info in cols:
                df.loc[idx, info] = driver.find_element(By.XPATH, xpath).text
            else:
                df.insert(len(df.columns), info, driver.find_element(By.XPATH, xpath).text)            
    except:
        err_list.append(info)
            

    # URL 수집
    df.loc[idx, 'URL'] = driver.current_url
    
    if len(err_list) != 0:
        print(f"Error: '{disease}'의 '{err_list}' 정보 수집 실패")
    else:
        print(f"'{disease}'의 정보 수집 완료")
    driver.close()

In [19]:
import pandas as pd
# 질환 20개 리스트
diseases = ['과민성 대장 증후군', '당뇨병', '심부전', '천식', '급성대장염',
           '감기', '급성 경막하 출혈', '인플루엔자', '긴장성 두통', '고산병',
           '아나필락시스', '루게릭병', '장폐색', '아토피 피부염', '대상포진',
           '돌발성 난청', '만성 비염', '신우신염', '뇌전증', '패혈증']

snu_df = pd.DataFrame(columns=['질환명', '영문명', '증상 키워드', '정의', '증상', '원인', '관련신체기관',
                               '진단', '검사', '치료', '경과/합병증', '생활가이드', 'URL'])

for idx in range(len(diseases)):
    disease = diseases[idx]
    snu_crowling(disease, idx, snu_df)

'과민성 대장 증후군'의 정보 수집 완료
'당뇨병'의 정보 수집 완료
'심부전'의 정보 수집 완료
'천식'의 정보 수집 완료
'급성대장염'의 정보 수집 완료
'감기'의 정보 수집 완료
'급성 경막하 출혈'의 정보 수집 완료
'인플루엔자'의 정보 수집 완료
'긴장성 두통'의 정보 수집 완료
'고산병'의 정보 수집 완료
'아나필락시스'의 정보 수집 완료
'루게릭병'의 정보 수집 완료
'장폐색'의 정보 수집 완료
'아토피 피부염'의 정보 수집 완료
'대상포진'의 정보 수집 완료
'돌발성 난청'의 정보 수집 완료
'만성 비염'의 정보 수집 완료
'신우신염'의 정보 수집 완료
'뇌전증'의 정보 수집 완료
'패혈증'의 정보 수집 완료


In [27]:
snu_df.head(3)

,질환명,영문명,증상 키워드,정의,증상,원인,관련신체기관,진단,치료,URL
0,과민성 대장 증후군,irritable bowel syndrome,"두통 , 복통 , 위경련 , 배 속 가스 , 변비 , 설사",과민성 대장 증후군은 대장 내시경이나 엑스선검사로 확인되는 특정 질환은 없지만 식사...,전형적인 증상으로는 복통과 배변 습관의 변화를 들 수 있는데 복통이 심하더라도 배변...,"명확한 원인은 아직 밝혀진 것이 없으며, 내장 감각의 과민성 증가, 위장관 운동성의...",대장,"과민성 대장 증후군은 생화학적, 구조적 이상으로 설명할 수 없으므로 어떤 한 가지 ...",치료는 과민성 대장 증후군의 원인이 되는 심리적 불안과 갈등을 제거하는 것이 가장 ...,http://www.snuh.org/health/nMedInfo/nView.do?c...
1,당뇨병,diabetes mellitus,"무감각증 , 소양감 , 변비 , 저혈압 , 거품뇨 , 단백뇨",당뇨병은 인슐린의 분비량이 부족하거나 정상적인 기능이 이루어지지 않는 등의 대사질환...,약한 고혈당에서는 대부분의 환자들이 증상을 느끼지 못하거나 모호해서 당뇨병이라고 생...,"당뇨병은 제1형과 제2형으로 구분되는데, 제1형 당뇨병은 이전에 '소아 당뇨병'이라...",췌장,혈액검사로 진단한다. 증상이 없는 경우 8시간 이상 금식 후에 측정한 혈당이 126...,제1형 당뇨병의 경우에는 인슐린 치료가 필요하다. 제2형 당뇨병의 경우에는 생활 습...,http://www.snuh.org/health/nMedInfo/nView.do?c...
2,심부전,heart failure,"기침 , 피로 , 호흡장애 , 부종",심부전이란 심장의 구조적 또는 기능적 이상으로 인해 심장이 혈액을 받아들이는 충만 ...,가장 흔하고 중요한 증상은 숨이 차는 것(호흡곤란)이다. 계단을 오르기만 해도 과도...,"매우 다양한 원인에 의해 심부전이 초래할 수 있는데, 심장 혈관(관상동맥) 질환(예...",심장,"호흡곤란, 피로감 등과 같은 심부전의 주관적 증상과 진찰 및 청진 시 이상 징후, ...",급성이냐 만성이냐에 따라 치료 방침은 다르다. 급성 심부전은 주로 1주일 이내 갑자...,http://www.snuh.org/health/nMedInfo/nView.do?c...


In [30]:
snu_df.to_csv('snuh_crowling_ver_2.csv', index=False, encoding='utf-8-sig')

### 서울아산병원 데이터 크롤링 [(링크)](https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseSubmain.do)

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


def asan_crowling(disease, disease_en, idx, df):
    chrome_option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_option)

    # 서울아산병원 홈페이지
    driver.get('https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseSubmain.do')

    search_ipt = driver.find_element(By.XPATH, '//*[@id="regionSrc"]')
    search_ipt.send_keys(disease)
    search_ipt.send_keys(Keys.RETURN)
    time.sleep(1)
    
    
    
    if "없습니다" in driver.find_element(By.CLASS_NAME, 'listCont').text:
        search_ipt = driver.find_element(By.XPATH, '//*[@id="regionSrc"]')
        search_ipt.clear()
        search_ipt.send_keys(disease_en)
        search_ipt.send_keys(Keys.RETURN)
        time.sleep(1)


    results_list = driver.find_element(By.CLASS_NAME, 'descBox').find_elements(By.TAG_NAME, 'li')
    elem_list = []
    title_list = []
    syn_list = []
    for result in results_list:
        title_space = result.find_element(By.CLASS_NAME, 'contTitle')
        elem_list.append(title_space.find_element(By.TAG_NAME, 'a'))
        title_list.append(title_space.find_element(By.TAG_NAME, 'a').text.replace(" ", "").split('(')[0])
        title_space = driver.find_element(By.CLASS_NAME, 'contBox').find_elements(By.TAG_NAME, 'dd')

        if len(title_space) == 4:
            syn_list.append(result.find_element(By.CSS_SELECTOR, 'dd:nth-child(8)').text.replace(" ", '').split(','))
        else:
            syn_list.append([""])

    best = []
    for i in range(len(title_list)):
        title = title_list[i]
        syn = syn_list[i]            
            
        if disease.replace(" ", "") == title:
            best.insert(0, title_list.index(title))
        elif disease.replace(" ", "") in syn:
            best.insert(0, title_list.index(title))
        elif disease.replace(" ", "") in title:
            best.append(title_list.index(title))

        
    elem_list[best[0]].click()


    description = driver.find_element(By.CLASS_NAME, 'descDl').find_elements(By.TAG_NAME, 'dt')
    desc_list = [desc.text for desc in description]

    # 질환 세부 정보 페이지의 목차와 xpath를 매칭
    xpath_dict = dict.fromkeys(desc_list)
    
    for i in range(len(desc_list)):
        xpath_dict[desc_list[i]] = f'//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[{i+1}]'
        
    cols = list(df.columns)



    # 질환명은 따로 수집하여, 파싱을 통해 국문명과 영문명을 추출
    name = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/strong').text
    kr_name = name.split('(')[0]
    en_name = name.split('(')[1].replace(')','')
    df.loc[idx, '질환명'] = kr_name
    df.loc[idx, '영문명'] = en_name


    # 동의어 수집
    df.loc[idx, '동의어'] = str(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/dl/dd[4]').text.split(','))
    

    # 증상 키워드 수집
    df.loc[idx, '증상 키워드'] = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/dl/dd[1]').text
    
    
    # 질환 세부 정보 수집
    err_list = []
    try:
        for info, xpath in xpath_dict.items():
            if info in cols:
                df.loc[idx, info] = driver.find_element(By.XPATH, xpath).text
            else:
                df.insert(len(df.columns), info, None)
                df.loc[idx, info] = driver.find_element(By.XPATH, xpath).text            
    except:
        err_list.append(info)
    
    # URL 수집
    df.loc[idx, 'URL'] = driver.current_url

    if len(err_list) != 0:
        print(f"Error: '{disease}'의 '{err_list}' 정보 수집 실패")
    else:
        print(f"'{disease}'의 정보 수집 완료")
    driver.close()

In [24]:
import pandas as pd
snu_df = pd.read_csv('snuh_crowling.csv')
disease_kr = snu_df['질환명'].tolist()
disease_en = snu_df['영문명'].tolist()
disease_dict = dict.fromkeys(disease_kr)
for i in range(len(disease_kr)):
    disease_dict[disease_kr[i]] = disease_en[i]
    
# 질환 20개 리스트
diseases = disease_kr

asan_df = pd.DataFrame(columns=['질환명', '영문명', '동의어', '증상 키워드', '정의', '증상', '원인', '진단', '치료', '경과', '주의사항', 'URL'])

for idx in range(len(diseases)):
    disease = diseases[idx]
    asan_crowling(disease, disease_dict[disease], idx, asan_df)

'과민성 대장 증후군'의 정보 수집 완료
'당뇨병'의 정보 수집 완료
'심부전'의 정보 수집 완료
'천식'의 정보 수집 완료
'급성 대장염'의 정보 수집 완료
'감기'의 정보 수집 완료
'급성 경막하 출혈'의 정보 수집 완료
'인플루엔자'의 정보 수집 완료
'긴장성 두통'의 정보 수집 완료
'고산병'의 정보 수집 완료
'아나필락시스'의 정보 수집 완료
'루게릭병'의 정보 수집 완료
'장폐색'의 정보 수집 완료
'아토피 피부염'의 정보 수집 완료
'대상포진'의 정보 수집 완료
'돌발성 난청'의 정보 수집 완료
'만성 비염'의 정보 수집 완료
'급성 신우신염'의 정보 수집 완료
'뇌전증'의 정보 수집 완료
'패혈증'의 정보 수집 완료


In [26]:
asan_df.head(3)

,질환명,영문명,동의어,증상 키워드,정의,증상,원인,진단,치료,경과,주의사항,URL,FAQ
0,자극성 장 증후군,Irritable bowel syndrome,"['과민성대장', '과민성대장증후군', '과민성장증후군', '과민성창자', '과민성...","복부 통증, 설사, 체중감소, 점액변, 변비",자극성 장 증후군은 특별한 기질적 원인이 확인되지 않는 상태에서 만성적으로 지속되는...,대장은 정상적으로 하루 1회 정도 배변을 유도하도록 움직입니다. 그러나 대장이 과민...,아직까지 자극성 장 증후군의 원인은 명확히 밝혀지지 않았습니다. 유전적 가족적 요인...,자극성 장 증후군은 구조적 이상으로 설명할 수 없으므로 특정 검사 방법으로 진단하기...,자극성 장 증후군의 원인이 되는 심리적 불안과 갈등을 제거하는 것이 가장 중요합니다...,암이나 환자의 정신과적인 이상 소견(인격 이상 등)과 관련이 있습니다. 감별이 필요...,"저지방식, 저자극식을 원칙으로 식사합니다. 지방성 음식, 향신료가 많이 들어간 음식...",https://www.amc.seoul.kr/asan/healthinfo/disea...,None
1,당뇨병,Diabetes mellitus,['당뇨'],"고혈당, 소양감, 체중감소, 피로감, 시야장애, 다뇨, 저림, 다음, 다식",당뇨병은 소변으로 포도당이 배출된다고 하여 이름 붙여진 병입니다. 정상인의 경우 소...,"당뇨병에 걸리면 소변으로 포도당이 빠져나가는데, 이때 수분을 같이 끌고 나가기 때문...","당뇨병의 발생에는 유전과 환경이 중요한 역할을 합니다. 즉, 당뇨병은 이것에 걸리기...",당뇨병은 혈당을 측정하여 진단합니다. 이때 혈당 검사란 손가락 끝에서 채혈을 하는 ...,"당뇨병의 치료 방법으로는 식사 요법, 운동 요법, 약물 치료가 있습니다. 가벼운 당...",당뇨병의 합병증에는 급성 대사성 합병증과 만성 합병증이 있습니다. 급성 대사성 합병...,"당뇨병을 예방하기 위해서는 일단 당뇨병을 유발할 수 있는 환경 인자인 비만, 좌식 ...",https://www.amc.seoul.kr/asan/healthinfo/disea...,None
2,울혈성 심부전,Congestive heart failure,"['울혈성심부전증', '울혈심부전', '울혈심부전증']","빈맥, 심부전, 호흡곤란, 울혈성 심부전, 소변량 감소, 야뇨증, 두통, 불면증, 청색증",울혈성 심부전은 여러 원인으로 인해 심장이 신체 조직이나 기관에서 필요한 혈액(특히...,"울혈성 심부전의 일반적인 증상으로는 전반적으로 몸이 약해짐, 피곤함, 운동을 조금만...",나이가 들면 심장의 기능이 감소하여 울혈성 심부전이 발생할 위험성이 커집니다. 65...,"울혈성 심부전은 임상 증상, 신체 검사, 간단한 검사(심전도, 흉부 X-ray 촬영...","1. 약물 요법\n 1) 일반적인 원칙\n약물 요법은 이뇨제, 안지오텐신 계열 차...",울혈성 심부전은 여성보다 남성에게 더 흔하게 나타납니다. 사망률은 심부전의 발생 원...,디지탈리스의 혈중 치료 농도는 일반적으로 1~2mcg/mL입니다. 일반적으로 2mc...,https://www.amc.seoul.kr/asan/healthinfo/disea...,None


In [28]:
asan_df.to_csv('asan_crowling_ver_2.csv', index=False, encoding='utf-8-sig')